This script was used to calculate the matrices of the inverse of the local resolution for DIVAnd. Some issues were encountered when trying to create netCDF files in Julia:
* NCDatasets: Couldn't find a way to assign data to the lon and lat dims, so the netCDF file produced using this package has lon and lat dims with lengths but no contents
* NetCDF: Couldn't figure out how to write data to the variables I defined or how to close a dataset object once opened. Also couldn't find documentation on how to write multiple variables to a newly created netCDF object; there were only examples for creating single-variable netCDF files

In [1]:
using DIVAnd
using CSV
using NCDatasets
using NetCDF
using DataStructures

In [2]:
# Calculate the pmn for 0m for all combinations of year/season
# Use these pmns for all other depths
mask_dir = string("C:\\Users\\HourstonH\\Documents\\NEP_climatology\\data\\",
    "value_vs_depth\\16_diva_analysis\\masks\\")

pmn_dir = string("C:\\Users\\HourstonH\\Documents\\NEP_climatology\\data\\",
    "value_vs_depth\\16_diva_analysis\\pmn\\")

# Use same pmn for all files since they all use the same mask

"C:\\Users\\HourstonH\\Documents\\NEP_climatology\\data\\value_vs_depth\\16_diva_analysis\\pmn\\"

In [3]:
# Read in mask
mask_filename = string(mask_dir, "Oxy_0m_1991_JFM_mask_6min.nc")

mask_ds = Dataset(mask_filename)

Lon2d, Lat2d = ndgrid(mask_ds["lon"], mask_ds["lat"])

# Compute inverse of resolution
pm, pn = DIVAnd_metric(Lon2d, Lat2d)

([0.0024923218319040544 0.0024924265561150263 … 0.004315922292037964 0.004316465456226367; 0.0024923219353162737 0.002492426607209966 … 0.004315922200753765 0.004316465727327514; … ; 0.0024923219353162737 0.002492426607209966 … 0.004315922200753765 0.004316465727327514; 0.0024923218319040544 0.0024924265561150263 … 0.004315922292037964 0.004316465456226367], [0.0021583688291264883 0.002158368805044173 … 0.002158368805044173 0.00215836878381549; 0.0021583688291264883 0.002158368805044173 … 0.002158368805044173 0.00215836878381549; … ; 0.0021583688291264883 0.002158368805044173 … 0.002158368805044173 0.00215836878381549; 0.0021583688291264883 0.002158368805044173 … 0.002158368805044173 0.00215836878381549])

In [12]:
Lon2d

10800×7200 Matrix{Float64}:
 -159.998  -159.998  -159.998  -159.998  …  -159.998  -159.998  -159.998
 -159.994  -159.994  -159.994  -159.994     -159.994  -159.994  -159.994
 -159.99   -159.99   -159.99   -159.99      -159.99   -159.99   -159.99
 -159.985  -159.985  -159.985  -159.985     -159.985  -159.985  -159.985
 -159.981  -159.981  -159.981  -159.981     -159.981  -159.981  -159.981
 -159.977  -159.977  -159.977  -159.977  …  -159.977  -159.977  -159.977
 -159.973  -159.973  -159.973  -159.973     -159.973  -159.973  -159.973
 -159.969  -159.969  -159.969  -159.969     -159.969  -159.969  -159.969
 -159.965  -159.965  -159.965  -159.965     -159.965  -159.965  -159.965
 -159.96   -159.96   -159.96   -159.96      -159.96   -159.96   -159.96
 -159.956  -159.956  -159.956  -159.956  …  -159.956  -159.956  -159.956
 -159.952  -159.952  -159.952  -159.952     -159.952  -159.952  -159.952
 -159.948  -159.948  -159.948  -159.948     -159.948  -159.948  -159.948
    ⋮                    

In [5]:
# Export pmn in netCDF file
pmn_ds_filename = string(pmn_dir, "divand_pmn_for_mask_6min_v2.nc")
pmn_ds = Dataset(pmn_ds_filename, "c")  # c stands for create new file

NCDataset: C:\Users\HourstonH\Documents\NEP_climatology\data\value_vs_depth\16_diva_analysis\pmn\divand_pmn_for_mask_6min_v2.nc
Group: /



In [7]:
# Define lat and lon dims
lon_dim = defDim(pmn_ds, "lon", length(mask_ds["lon"]))
lat_dim = defDim(pmn_ds, "lat", length(mask_ds["lat"]))

# # Dictionaries of attributes
# lon_atts = Dict("longname" => "Longitude", "units" => "degrees east")
# lat_atts = Dict("longname" => "Latitude", "units" => "degrees north")
# pm_atts = Dict(
#     "longname" => "Inverse of the local resolution in the x (longitude) dimension using the mean Earth radius", 
#     "units" => "meters")
# pn_atts = Dict(
#     "longname" => "Inverse of the local resolution in the y (latitude) dimension using the mean Earth radius",
#     "units" => "meters")

# lon_dim = NcDim("lon", mask_ds["lon"], lon_atts)
# lat_dim = NcDim("lat", mask_ds["lat"], lat_atts)
# pm_var = NcVar("pm", [lon_dim, lat_dim]; atts=pm_atts, t=Float32)
# pn_var = NcVar("pn", [lon_dim, lat_dim]; atts=pn_atts, t=Float32)

# Define variables pm and pn
pm_var = defVar(pmn_ds, "pm", Float64, ("lon", "lat"), 
    attrib=OrderedDict(
        "longname" => "Inverse of the local resolution in the x (longitude) dimension using the mean Earth radius",
        "units" => "meters"))
pn_var = defVar(pmn_ds, "pn", Float64, ("lon", "lat"),
    attrib=OrderedDict(
        "longname" => "Inverse of the local resolution in the y (latitude) dimension using the mean Earth radius",
        "units" => "meters"))

pn (10800 × 7200)
  Datatype:    Float64
  Dimensions:  lon × lat
  Attributes:
   longname             = Inverse of the local resolution in the y (latitude) dimension using the mean Earth radius
   units                = meters


In [8]:
# How to add data to the dims?
# lon_dim[:] = mask_ds["lon"][:]
# lat_dim[:] = mask_ds["lat"][:]

# NetCDF.create(pmn_ds_filename, [pm, pn]) do nc
#       # Writing data to the file is done using putvar
#     NetCDF.putvar(nc, "pm", pm)
#     NetCDF.putvar(nc, "pn", pn)
# end

# NetCDF.open(pmn_ds_filename, mode=NC_WRITE) do nc
#     # Writing data to the file is done using putvar
#      NetCDF.putvar(nc, "pm", pm)
# #     NetCDF.putvar(nc, "pn", pn)
# end

# Open it again to write another var (pn)
# NetCDF.open(pmn_ds_filename, mode=NC_WRITE) do nc
#     NetCDF.putvar(nc, "pn", pn)
# end

# Writing data to the file is done using putvar
# ncout = NetCDF.putvar(nc, "pm", pm)
# ncout = NetCDF.putvar(nc, "pn", pn)

LoadError: MethodError: no method matching setindex!(::Nothing, ::Vector{Union{Missing, Float64}}, ::Colon)

In [9]:
# Assign data to pm and pn vars
pm_var[:,:] = pm
pn_var[:,:] = pn

10800×7200 Matrix{Float64}:
 0.00215837  0.00215837  0.00215837  …  0.00215837  0.00215837  0.00215837
 0.00215837  0.00215837  0.00215837     0.00215837  0.00215837  0.00215837
 0.00215837  0.00215837  0.00215837     0.00215837  0.00215837  0.00215837
 0.00215837  0.00215837  0.00215837     0.00215837  0.00215837  0.00215837
 0.00215837  0.00215837  0.00215837     0.00215837  0.00215837  0.00215837
 0.00215837  0.00215837  0.00215837  …  0.00215837  0.00215837  0.00215837
 0.00215837  0.00215837  0.00215837     0.00215837  0.00215837  0.00215837
 0.00215837  0.00215837  0.00215837     0.00215837  0.00215837  0.00215837
 0.00215837  0.00215837  0.00215837     0.00215837  0.00215837  0.00215837
 0.00215837  0.00215837  0.00215837     0.00215837  0.00215837  0.00215837
 0.00215837  0.00215837  0.00215837  …  0.00215837  0.00215837  0.00215837
 0.00215837  0.00215837  0.00215837     0.00215837  0.00215837  0.00215837
 0.00215837  0.00215837  0.00215837     0.00215837  0.00215837  0.002158

In [16]:
# Close netCDF file
close(pmn_ds)

# NetCDF.close(ncout)

closed NetCDF NCDataset

Add lon and lat 2d mesh grid data to the pmn netCDF file as an afterthought, since the lon and lat dims are not filled with the right data.

In [17]:
# Add meshgrid lon and lat to the pmn netCDF file
pmn_ds_filename = string(pmn_dir, "divand_pmn_for_mask_6min_v2.nc")

pmn_ds = Dataset(pmn_ds_filename, "a")

NCDataset: C:\Users\HourstonH\Documents\NEP_climatology\data\value_vs_depth\16_diva_analysis\pmn\divand_pmn_for_mask_6min_v2.nc
Group: /

Dimensions
   lon = 10800
   lat = 7200

Variables
  pm   (10800 × 7200)
    Datatype:    Float64
    Dimensions:  lon × lat
    Attributes:
     longname             = Inverse of the local resolution in the x (longitude) dimension using the mean Earth radius
     units                = meters

  pn   (10800 × 7200)
    Datatype:    Float64
    Dimensions:  lon × lat
    Attributes:
     longname             = Inverse of the local resolution in the y (latitude) dimension using the mean Earth radius
     units                = meters



In [19]:
lon2d_var = defVar(pmn_ds, "lon2d", Float64, ("lon", "lat"),
    attrib=OrderedDict("longname" => "Longitude", "units" => "degrees east"))

lat2d_var = defVar(pmn_ds, "lat2d", Float64, ("lon","lat"),
    attrib=OrderedDict("longname" => "Latitude", "units" => "degrees north"))

lat2d (10800 × 7200)
  Datatype:    Float64
  Dimensions:  lon × lat
  Attributes:
   longname             = Latitude
   units                = degrees north


In [20]:
lon2d_var[:,:] = Lon2d
lat2d_var[:,:] = Lat2d

10800×7200 Matrix{Float64}:
 30.0021  30.0062  30.0104  30.0146  …  59.9854  59.9896  59.9938  59.9979
 30.0021  30.0062  30.0104  30.0146     59.9854  59.9896  59.9938  59.9979
 30.0021  30.0062  30.0104  30.0146     59.9854  59.9896  59.9938  59.9979
 30.0021  30.0062  30.0104  30.0146     59.9854  59.9896  59.9938  59.9979
 30.0021  30.0062  30.0104  30.0146     59.9854  59.9896  59.9938  59.9979
 30.0021  30.0062  30.0104  30.0146  …  59.9854  59.9896  59.9938  59.9979
 30.0021  30.0062  30.0104  30.0146     59.9854  59.9896  59.9938  59.9979
 30.0021  30.0062  30.0104  30.0146     59.9854  59.9896  59.9938  59.9979
 30.0021  30.0062  30.0104  30.0146     59.9854  59.9896  59.9938  59.9979
 30.0021  30.0062  30.0104  30.0146     59.9854  59.9896  59.9938  59.9979
 30.0021  30.0062  30.0104  30.0146  …  59.9854  59.9896  59.9938  59.9979
 30.0021  30.0062  30.0104  30.0146     59.9854  59.9896  59.9938  59.9979
 30.0021  30.0062  30.0104  30.0146     59.9854  59.9896  59.9938  59.99

In [14]:
Lon2d[:,:]

10800×7200 Matrix{Float64}:
 -159.998  -159.998  -159.998  -159.998  …  -159.998  -159.998  -159.998
 -159.994  -159.994  -159.994  -159.994     -159.994  -159.994  -159.994
 -159.99   -159.99   -159.99   -159.99      -159.99   -159.99   -159.99
 -159.985  -159.985  -159.985  -159.985     -159.985  -159.985  -159.985
 -159.981  -159.981  -159.981  -159.981     -159.981  -159.981  -159.981
 -159.977  -159.977  -159.977  -159.977  …  -159.977  -159.977  -159.977
 -159.973  -159.973  -159.973  -159.973     -159.973  -159.973  -159.973
 -159.969  -159.969  -159.969  -159.969     -159.969  -159.969  -159.969
 -159.965  -159.965  -159.965  -159.965     -159.965  -159.965  -159.965
 -159.96   -159.96   -159.96   -159.96      -159.96   -159.96   -159.96
 -159.956  -159.956  -159.956  -159.956  …  -159.956  -159.956  -159.956
 -159.952  -159.952  -159.952  -159.952     -159.952  -159.952  -159.952
 -159.948  -159.948  -159.948  -159.948     -159.948  -159.948  -159.948
    ⋮                    

In [21]:
close(pmn_ds)

closed NetCDF NCDataset

## Testing code

In [20]:
lat, lon = ndgrid([30, 45, 60], [-160, -145, -130, -115])

([30 30 30 30; 45 45 45 45; 60 60 60 60], [-160 -145 -130 -115; -160 -145 -130 -115; -160 -145 -130 -115])

In [21]:
lon

3×4 Matrix{Int64}:
 -160  -145  -130  -115
 -160  -145  -130  -115
 -160  -145  -130  -115

In [22]:
lat

3×4 Matrix{Int64}:
 30  30  30  30
 45  45  45  45
 60  60  60  60

In [23]:
lat[1,:]

4-element Vector{Int64}:
 30
 30
 30
 30

In [24]:
lon[1,:]

4-element Vector{Int64}:
 -160
 -145
 -130
 -115

In [15]:
pmn_ds

NCDataset: C:\Users\HourstonH\Documents\NEP_climatology\data\value_vs_depth\16_diva_analysis\pmn\divand_pmn_for_mask_6min_v2.nc
Group: /

Dimensions
   lon = 10800
   lat = 7200

Variables
  pm   (10800 × 7200)
    Datatype:    Float64
    Dimensions:  lon × lat
    Attributes:
     longname             = Inverse of the local resolution in the x (longitude) dimension using the mean Earth radius
     units                = meters

  pn   (10800 × 7200)
    Datatype:    Float64
    Dimensions:  lon × lat
    Attributes:
     longname             = Inverse of the local resolution in the y (latitude) dimension using the mean Earth radius
     units                = meters

